In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import os

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define dataset paths
test_dir = r"C:\Black Book Project\HAM 10000\HAM10000_split\test"
valid_dir = r"C:\Black Book Project\HAM 10000\HAM10000_split\valid"

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet expects 224x224 images
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard for ResNet
])

# Load datasets
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=transform)

num_classes = len(test_dataset.classes)  # Assuming test & valid have the same class structure
print(f"Classes detected: {test_dataset.classes}")

# Create dataloaders
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

# Define the model (ResNet-50 with modified classifier)
model = models.resnet50(pretrained=True)  # Load pre-trained ResNet-50
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)  # Adjust last layer to match classes
model = model.to(device)

# Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training function
def train_model(model, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels).item()
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Acc: {epoch_acc:.4f}")

    print("Training complete!")
    return model

# Train the model
model = train_model(model, val_loader, criterion, optimizer, epochs=10)

# Save model
torch.save(model.state_dict(), "resnet50_skin_cancer.pth")
print("Model saved!")

# Risk Scoring Function
def risk_score(image_path, model, class_names):
    model.eval()
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    from PIL import Image
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(image)
        probs = torch.nn.functional.softmax(output, dim=1)
        score, pred = torch.max(probs, 1)
    
    return class_names[pred.item()], score.item()

# Test risk scoring
class_names = test_dataset.classes
test_image = r"C:\Black Book Project\HAM 10000\HAM10000_split\test\vasc\ISIC_0024370.jpg"
# Change to an actual image path
predicted_class, confidence = risk_score(test_image, model, class_names)
print(f"Predicted: {predicted_class}, Confidence: {confidence:.4f}")


Using device: cpu
Classes detected: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
Epoch 1/10 - Loss: 2.2178 - Acc: 0.0342
Epoch 2/10 - Loss: 1.8404 - Acc: 0.2360
Epoch 3/10 - Loss: 1.6801 - Acc: 0.4720
Epoch 4/10 - Loss: 1.5470 - Acc: 0.5031
Epoch 5/10 - Loss: 1.3894 - Acc: 0.6118
Epoch 6/10 - Loss: 1.2273 - Acc: 0.6894
Epoch 7/10 - Loss: 1.0962 - Acc: 0.7205
Epoch 8/10 - Loss: 0.9692 - Acc: 0.7733
Epoch 9/10 - Loss: 0.8956 - Acc: 0.7795
Epoch 10/10 - Loss: 0.7785 - Acc: 0.8665
Training complete!
Model saved!
Predicted: vasc, Confidence: 0.6162


In [3]:
import os
test_vasc_path = r"C:\Black Book Project\HAM 10000\HAM10000_split\test\vasc"
print("Available images:", os.listdir(test_vasc_path))


Available images: ['ISIC_0024370.jpg', 'ISIC_0024475.jpg', 'ISIC_0024662.jpg', 'ISIC_0024747.jpg', 'ISIC_0025197.jpg', 'ISIC_0025244.jpg', 'ISIC_0025250.jpg', 'ISIC_0025452.jpg', 'ISIC_0025578.jpg', 'ISIC_0025596.jpg', 'ISIC_0025599.jpg', 'ISIC_0025606.jpg', 'ISIC_0025612.jpg', 'ISIC_0025628.jpg', 'ISIC_0025677.jpg', 'ISIC_0025680.jpg', 'ISIC_0025707.jpg', 'ISIC_0025873.jpg', 'ISIC_0025924.jpg', 'ISIC_0026092.jpg', 'ISIC_0026336.jpg', 'ISIC_0026467.jpg', 'ISIC_0026490.jpg', 'ISIC_0027159.jpg', 'ISIC_0027210.jpg', 'ISIC_0027665.jpg', 'ISIC_0027672.jpg', 'ISIC_0027856.jpg', 'ISIC_0027888.jpg', 'ISIC_0028431.jpg', 'ISIC_0028714.jpg', 'ISIC_0028885.jpg', 'ISIC_0029404.jpg', 'ISIC_0029439.jpg', 'ISIC_0029448.jpg', 'ISIC_0029608.jpg', 'ISIC_0029742.jpg', 'ISIC_0030104.jpg', 'ISIC_0030606.jpg', 'ISIC_0030770.jpg', 'ISIC_0030882.jpg', 'ISIC_0030956.jpg', 'ISIC_0031093.jpg', 'ISIC_0031103.jpg', 'ISIC_0031201.jpg', 'ISIC_0031217.jpg', 'ISIC_0031346.jpg', 'ISIC_0031719.jpg', 'ISIC_0032076.jpg', '